# **Hands-on Validação**

---

Validação com sklearn 

Neste *hands on*, vamos aprofundar um pouco mais nos processos de validação usados pra medir a performance de algoritmos de machine learning. Aqui, vamos focar na tarefa de classificação como exemplo, mas nosso foco é no processo de validação. 

Vamos avaliar 3 tipos diferentes de validação e discutir vantagens e desvantagens de cada uma delas. Os tipos de validação são: 

* **Treino-Teste**: Esse tipo de validação consiste em separar, aleatoriamente, uma porção da base de dados para servir como conjunto de teste. Dessa forma, o algoritmo tem acesso somente a parte dos dados para construir o modelo e sua performance é avaliada no conjunto de dados que esteve escondido (teste) no processo. 

* **Validação Cruzada**: Neste tipo de validação, ao invés de separar em um conjunto fixo de treinamento e outro de teste, o processo é ampliado para todo dado disponível faça parte do treino e do teste em algum momento. Assim, definimos um número de partições para o dado e cada uma das partições é usada como teste em algum ponto. Ao final, consideramos a média das execuções para avaliar a performance do algoritmo.

* **Leave-one-out**: Este tipo de validação consiste em uma extensão da validação cruzada, em que se considera o número de partições igual à quantidade de amostras disponíveis no dado. Assim, cada conjunto de teste é sempre composto por uma única amostra do dado, ao passo que o restante das amostras é usada para treinamento do algoritmo.


É importante destacar que nosso interesse principal ao avaliar um algoritmo de machine learning é encontrar um algoritmo que tenha boa capacidade de **generalização**, ou seja, tem boa performance em dados que não foram vistos durante a etapa de treinamento. Todos os tipos de validação anterior, nos ajudam a avaliar essa capacidade, testando a solucão encontrada em um conjunto de dados **novo** para o algoritmo. Dentre os tipos de avaliação mencionados, tipicamente, a validação cruzada é mais amplamente usada, pois combina a maior parte das vantagens que veremos na sequência.

Mais informações sobre esses tipos de validação podem ser encontradas na documentação do sklearn, no link a seguir.

*   <a href = https://scikit-learn.org/stable/modules/cross_validation.html> Validação no sklearn </a>


Vamos utilizar a base de dados **diabetes.csv** </a> que pode ser encontrada no OpenML em: https://www.openml.org/d/37. Essa base descreve o problema de predizer o resultado positivo ou negativo de um teste para diabetes aplicado em um conjunto específico de pacientes na Índia. 


### *Importando* bibliotecas


In [2]:
import pandas as pd #biblioteca para manipulação de dados
import numpy as np #biblioteca para utilizacao de vetores e matrizes
import matplotlib.pyplot as plt #bibloteca para plotar graficos

In [3]:
#liberando acesso do colab aos arquivos no drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Carregamento dos dados



In [43]:
#lendo o csv que contem a base de dados e armazanando em um df
df = pd.read_csv('/content/gdrive/My Drive/DADOS/diabetes.csv')

In [44]:
#imprimindo as 5 primeiras linhas do df para confirmação
df.head(5)

,preg,plas,pres,skin,insu,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,tested_positive
1,1,85,66,29,0,26.6,0.351,31,tested_negative
2,8,183,64,0,0,23.3,0.672,32,tested_positive
3,1,89,66,23,94,28.1,0.167,21,tested_negative
4,0,137,40,35,168,43.1,2.288,33,tested_positive


*   preg: number of times pregnant
*   plas: Plasma glucose concentration a 2 hours in an oral glucose tolerance test
*   pres: Diastolic blood pressure (mm Hg)
*   skin: Triceps skin fold thickness (mm)
*   insu: 2-Hour serum insulin (mu U/ml)
*   mass: Body mass index (weight in kg/(height in m)^2)
*   pedi: Diabetes pedigree function
*   age: Age (years)
*   class: Class variable (tested_positive or tested_negative)

In [45]:
# Verificando o numero de amostras (linhas) e features (colunas) do dataset. 
print('Amostras e Features:', df.shape)

Amostras e Features: (768, 9)


In [46]:
# Verificando quais são os tipos das features   #### O CERTO QUAL É O NOME DAS COLUNAS(FEATURES)
print('\n',df.columns,'\n')  #### PROFESSOR SOMENTE LISTA OS NOMES DAS COLUNAS

df.info()  ################ EUDARDO MOSTRANDO OS TIPOS DE FEATURES, TIPOS DOS DADOS DE CADA COLUNA.


 Index(['preg', 'plas', 'pres', 'skin', 'insu', 'mass', 'pedi', 'age', 'class'], dtype='object') 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   preg    768 non-null    int64  
 1   plas    768 non-null    int64  
 2   pres    768 non-null    int64  
 3   skin    768 non-null    int64  
 4   insu    768 non-null    int64  
 5   mass    768 non-null    float64
 6   pedi    768 non-null    float64
 7   age     768 non-null    int64  
 8   class   768 non-null    object 
dtypes: float64(2), int64(6), object(1)
memory usage: 54.1+ KB


In [47]:

a = dict(df.dtypes.value_counts())  ######## contar a quantidade e tipos
a

{dtype('int64'): 6, dtype('float64'): 2, dtype('O'): 1}

### Pré processamento

Nesse problema, o único atributo categórico é a classe. Alguns modelos podem ter dificuldade de lidar com esse atributo. Nesse caso, precisamos transformar os valores de "tested_negative" e "tested_positivo" para 0 e 1.

Para isso, vamos fazer uma função bem simples. 

In [48]:
#criando um dicionario de dados para o mapeamento
name_to_class = {
    'tested_negative': 0,
    'tested_positive': 1
}

#substituindo os valores categóricos pelo mapeamento
df['class'] = df['class'].map(name_to_class)

#check
df.head(5)

,preg,plas,pres,skin,insu,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Uma outra etapa importante do pré-processamento consiste na avaliação de dados faltantes. 



In [49]:
# Analisando o resumo da base
df.describe()

,preg,plas,pres,skin,insu,mass,pedi,age,class
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


Os modelos implementados no sklearn recebem como entrada para a modelagam um ou mais arrays. Dessa forma, precisamos modificar o df original para que seja possível a modelagem correta. 

Para isso, vamos separar o label das amostras, armazenar o nome das features - já que os arrays não fazem isso - e depois retirar a coluna de labels do df original. Em seguida, vamos converter o df para array usando o numpy!

In [50]:
################## EDUARDO MELHORANDO A SITUAÇÃO - sem deletar ou motificar o DF

labels_edu = np.array(df['class'])
data_edu =   np.array(df.drop('class',axis=1))
print(labels_edu)
print(data_edu)

[1 0 1 0 1 0 1 0 1 1 0 1 0 1 1 1 1 1 0 1 0 0 1 1 1 1 1 0 0 0 0 1 0 0 0 0 0
 1 1 1 0 0 0 1 0 1 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 1 0 1 0 0 0 1 0 1 0
 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1
 1 0 0 1 1 1 0 0 0 1 0 0 0 1 1 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 1 0 1 1 0 0 0 1 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 1 0 1 0 1 0 0 0 0 0
 1 1 1 1 1 0 0 1 1 0 1 0 1 1 1 0 0 0 0 0 0 1 1 0 1 0 0 0 1 1 1 1 0 1 1 1 1
 0 0 0 0 0 1 0 0 1 1 0 0 0 1 1 1 1 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0
 1 0 1 0 0 1 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 0 1 1 0 0 1 0 0 0 1 1 1 0 0
 1 0 1 0 1 1 0 1 0 0 1 0 1 1 0 0 1 0 1 0 0 1 0 1 0 1 1 1 0 0 1 0 1 0 0 0 1
 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 1 0 1 1 0 0 1 0 0 1 0 0 1
 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 1 0 0 1 0 0 1 0 1 1 0 1 0 1 0 1
 0 1 1 0 0 0 0 1 1 0 1 0 1 0 0 0 0 1 1 0 1 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 1
 1 1 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1
 0 0 0 1 1 0 0 0 0 0 0 0 

In [51]:
# armazenando os labels em um array
labels = np.array(df['class'])

# salvando a ordem das features
feature_list = list(df.columns)

In [52]:
# removendo a coluna de labels do df original
df = df.drop('class', axis = 1)

# check
df.columns

Index(['preg', 'plas', 'pres', 'skin', 'insu', 'mass', 'pedi', 'age'], dtype='object')

In [53]:
# convertendo df para array
data = np.array(df)

### Treino-Teste

Em uma das aulas anteriores já passamos por essa etapa, porém com menos detalhes. Agora, vamos avaliar com um pouco mais de cuidado as estratégias de validação. 

Para usar a a estratégia de validação *treino-teste*, precisamos apenas separar uma parte dos nossos dados para conjunto de treino e a parte restante para conjunto de teste. 

Para fazer essa separação, vamos usar a função <a href = http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html> train_test_split</a> do sklearn.


In [54]:
# importar train_test_split do scikitlearn 
from sklearn.model_selection import train_test_split

In [55]:
# aplicando a funcao train_test_split para separar os conjuntos de treino e 
# teste segundo uma porcentagem de separação definida. 
train_data1, test_data1, train_labels1, test_labels1 = train_test_split(data, labels, test_size = 0.25, random_state = 42)

In [56]:
train_data2, test_data2, train_labels2, test_labels2 = train_test_split(data, labels, test_size = 0.25, random_state = 123)

In [58]:
train_data3, test_data3, train_labels3, test_labels3 = train_test_split(data, labels, test_size = 0.35, random_state = 42)  ### 35%

### Modegalem com Treino-Teste

Não existe uma regra pré-definida para o tamanho do conjunto de teste. Tipicamente usamos entre 20% e 40% da base de dados, dependendo do número total de amostras disponíveis. 

Após separar a base de dados em treino e teste, o próximo passo é executar o modelo escolhido e avaliar o resultado. 

Nesse momento estamos menos preocupados em comparar algoritmos distintos para escolher o melhor mas sim em entender a melhor estratégia para montar o processo de validação. Assim, vamos aplicar somente um dos algoritmos que vimos anteriormente e focar nos resultados que podemos obter com os diferentes métodos de validação.  


# Random Forest Classifiers


Vamos agora modelar o nosso problema utilizando o Random Forest Classifier!

Esse modelo é um dos mais utilizados tanto na sua versão de regressor quanto para sua versão de classificador e, em geral, apresenta ótimos resultados!

In [59]:
# importar o modelo Random Forest Regressor
from sklearn.ensemble import RandomForestClassifier

# treinando o modelo 
classifier1 = RandomForestClassifier(n_estimators= 10, random_state=42).fit(train_data1, train_labels1)  ### 10 arvores dentro da floresta
classifier2 = RandomForestClassifier(n_estimators= 10, random_state=42).fit(train_data2, train_labels2)
classifier3 = RandomForestClassifier(n_estimators= 10, random_state=42).fit(train_data3, train_labels3)

In [61]:
# aplicando o modelo treinado para a previsão do resultado do teste
predictions1_labels = classifier1.predict(test_data1)
predictions2_labels = classifier2.predict(test_data2)
predictions3_labels = classifier3.predict(test_data3)

# Exibindo dataframe com valores 10 reais e suas respectivas previsões
p = pd.DataFrame({'Real': test_labels3, 'Previsto': predictions3_labels})  
p.head(10)

,Real,Previsto
0,0,0
1,0,0
2,0,0
3,0,0
4,0,1
5,0,0
6,0,0
7,0,1
8,0,1
9,0,1


In [ ]:
print(predictions1_labels)
print(predictions2_labels)

[0 0 0 0 0 1 0 1 1 1 0 1 1 0 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1 1 1 1 1 1 1
 1 0 1 0 0 1 1 0 1 1 0 0 1 0 1 1 0 0 0 1 0 0 1 1 0 0 1 0 1 0 0 0 1 1 0 0 0
 0 0 0 0 0 0 1 0 0 1 1 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 1 0 1 0 0 1 0 1 0
 1 0 1 0 1 1 0 0 0 0 0 0 0 0 1 0 1 1 1 0 1 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 0 0 1 0 0 0 1 0 0 1 0 0 1
 0 0 0 1 1 0 0]
[1 0 1 0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 1 1 0 0 0 1
 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 1 1 1 0 1 0 0 0 1 0 0 0 0 1 1 0
 1 0 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 1 0 0 0 0 1 1 0 0 0 1 0 0 0 1 0 1 0 0 1
 0 1 1 0 0 0 1 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 1 1
 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 1 1 0 0 0 0 0 1 1 0 1 1 0 1
 1 0 1 0 0 1 0]


In [ ]:
# importar biblioteca para calculo de métricas
from sklearn import metrics 

print('\nAcurácia\n', metrics.accuracy_score(test_labels1, predictions1_labels)) 
print('\nAcurácia\n', metrics.accuracy_score(test_labels2, predictions2_labels)) 
print('\nAcurácia\n', metrics.accuracy_score(test_labels3, predictions3_labels)) 


Acurácia
 0.7395833333333334

Acurácia
 0.7447916666666666

Acurácia
 0.758364312267658


Aqui podemos observar algumas coisas a respeito dessa estratégia de validação. 

Os dois primeiros conjuntos de teste possuem exatamente o mesmo tamanho (25% da base de dados), mas os resultados de acúracia foram diferentes. O segundo conjunto teve uma acurácia superior. A única diferença entre os dois conjuntos de teste foi a aleatoriedade na seleção. 

Aqui fica muito claro que este método é muito sujeito ao viés de seleção. É difícil dizer que o segundo modelo é melhor que o primeiro, apesar do resultado superior. Isso acontece porque esse resultado pode ter sido fruto somente de uma seleção mais favorável. De maneira informal, podemos pensar como se o conjunto de teste, aleatoriamente escolhido para o segundo conjunto de dados, possuía as amostras mais fáceis de serem acertadas. 

Além disso, observamos também que o terceiro conjunto obteve um resultado melhor que os anteriores. Aqui também nossa capacidade de tomar uma decisão é limitada pelo método. O nosso terceiro conjunto de teste tinha tamanho 30% da base de dados. Um primeiro pensamento aqui poderia ser um resultado pior, já que a base de treinamento acabaria sendo menor, tendo um teste um pouco maior. Porém, estamos sujeitos novamente ao viés da seleção, por mais que ela seja aleatória. Ainda assim, podemos ver outro ponto negativo dessa estratégia, que é: quanto maior nosso dado de teste, menos dados temos para a modelagem. 

Você consegue pensar em uma situação em que essa estratégia pode ser muito ruim? R.: Dados desbalanceados!

Na verdade, essa estratégia raramente é usada para se tirar uma conclusão sobre a qualidade do modelo. No fim, ela acaba sendo só um primeiro passo para construir as estratégias mais elaboradas de validação que veremos a seguir.

Antes porém, vamos olhar também as outras métricas que já conhecemos. Faça o exercício de comparar os resultados entre os três conjuntos

In [62]:
# importar biblioteca para calculo de métricas
from sklearn import metrics 

#avaliando o modelo
print('Matriz de Confusão\n', metrics.confusion_matrix(test_labels1, predictions1_labels)) 
print('Matriz de Confusão\n', metrics.confusion_matrix(test_labels2, predictions2_labels)) 
print('Matriz de Confusão\n', metrics.confusion_matrix(test_labels3, predictions3_labels)) 

print('\nAcurácia Balanceada por classe\n', metrics.balanced_accuracy_score(test_labels1, predictions1_labels)) 
print('\nAcurácia Balanceada por classe\n', metrics.balanced_accuracy_score(test_labels2, predictions2_labels)) 
print('\nAcurácia Balanceada por classe\n', metrics.balanced_accuracy_score(test_labels3, predictions3_labels)) 

print('\nPrecision\n', metrics.precision_score(test_labels1, predictions1_labels)) 
print('\nPrecision\n', metrics.precision_score(test_labels2, predictions2_labels)) 
print('\nPrecision\n', metrics.precision_score(test_labels3, predictions3_labels)) 

print('\nRecall\n', metrics.recall_score(test_labels1, predictions1_labels)) 
print('\nRecall\n', metrics.recall_score(test_labels2, predictions2_labels)) 
print('\nRecall\n', metrics.recall_score(test_labels3, predictions3_labels)) 

print('\nF1\n', metrics.f1_score(test_labels1, predictions1_labels)) 
print('\nF1\n', metrics.f1_score(test_labels2, predictions2_labels)) 
print('\nF1\n', metrics.f1_score(test_labels3, predictions3_labels)) 

print('\nAUCROC\n', metrics.roc_auc_score(test_labels1, predictions1_labels))
print('\nAUCROC\n', metrics.roc_auc_score(test_labels2, predictions2_labels))
print('\nAUCROC\n', metrics.roc_auc_score(test_labels3, predictions3_labels))

Matriz de Confusão
 [[97 26]
 [24 45]]
Matriz de Confusão
 [[100  19]
 [ 30  43]]
Matriz de Confusão
 [[153  28]
 [ 37  51]]

Acurácia Balanceada por classe
 0.72039589961117

Acurácia Balanceada por classe
 0.7146886151720963

Acurácia Balanceada por classe
 0.7124246609743847

Precision
 0.6338028169014085

Precision
 0.6935483870967742

Precision
 0.6455696202531646

Recall
 0.6521739130434783

Recall
 0.589041095890411

Recall
 0.5795454545454546

F1
 0.6428571428571428

F1
 0.6370370370370372

F1
 0.6107784431137726

AUCROC
 0.72039589961117

AUCROC
 0.7146886151720963

AUCROC
 0.7124246609743847


Resuminho...



In [63]:
print('\nClassification Report\n', metrics.classification_report(test_labels1, predictions1_labels)) 
print('\nClassification Report\n', metrics.classification_report(test_labels2, predictions2_labels)) 
print('\nClassification Report\n', metrics.classification_report(test_labels3, predictions3_labels)) 


Classification Report
               precision    recall  f1-score   support

           0       0.80      0.79      0.80       123
           1       0.63      0.65      0.64        69

    accuracy                           0.74       192
   macro avg       0.72      0.72      0.72       192
weighted avg       0.74      0.74      0.74       192


Classification Report
               precision    recall  f1-score   support

           0       0.77      0.84      0.80       119
           1       0.69      0.59      0.64        73

    accuracy                           0.74       192
   macro avg       0.73      0.71      0.72       192
weighted avg       0.74      0.74      0.74       192


Classification Report
               precision    recall  f1-score   support

           0       0.81      0.85      0.82       181
           1       0.65      0.58      0.61        88

    accuracy                           0.76       269
   macro avg       0.73      0.71      0.72       269
we

## Validação Cruzada

Agora voltamos nosso interesse para uma estratégia de validação um pouco mais complexa e útil, chamada de validação cruzada.

Vamos usar a mesma base de dados do exemplo anterior.

In [65]:
# lembre-se que já removemos a coluna com a classe anteriormente
df.head(5)

,preg,plas,pres,skin,insu,mass,pedi,age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


Na estratégia anterior, fizemos a separação do dado em conjunto de treinamento e conjunto de teste. Aqui não vamos fazer isso de maneira explícita. Vamos usar a função <a href = https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html> cross_val_score </a> para nos ajudar no processo. 

Essa função será responsável por controlar a separação dos conjuntos de treino e teste, além da iteração por cada um deles computando as métricas desejadas. 

Ainda podemos fazer essa iteração manualmente se for desejado, mas o uso da função simplifica o processo e otimiza algumas etapas, bastando para isso definir o número de conjuntos de validação que desejamos usar. 

Vamos fazer um primeiro exemplo na sequência

In [76]:
from sklearn.model_selection import cross_val_score

classifier_cv = RandomForestClassifier(n_estimators= 10, random_state=42) ### RadomForestClassifier

scores_cv = cross_val_score(classifier_cv, data, labels, cv=5)
scores_cv

array([0.74025974, 0.69480519, 0.77922078, 0.79738562, 0.74509804])

In [77]:
print("Acurácia: %0.2f (+/- %0.2f)" % (scores_cv.mean(), scores_cv.std() * 2))  ### desvio padrão pequeno, tudo ok

Acurácia: 0.75 (+/- 0.07)


Tipicamente, o score retornado é sempre o padrão do algoritmo escolhido. Para algoritmos de classificação, a grande maioria retorna por padrão a acurácia do algoritmo. 

Essa métrica de avaliação pode ser substituído conforme desejado, bastando passar para a função uma outra opção de score. Vamos usar mais algumas métricas que já conhecemos a seguir.

In [79]:
scores_cv_accuracy = cross_val_score(classifier_cv, data, labels, cv=5) ### eduardo
scores_cv_precision = cross_val_score(classifier_cv, data, labels, cv=5, scoring='precision')
scores_cv_recall = cross_val_score(classifier_cv, data, labels, cv=5, scoring='recall')
scores_cv_f1 = cross_val_score(classifier_cv, data, labels, cv=5, scoring='f1')

##### desvio padrão mostra quanto os dados estão distantes da média ###

print("Accuracy: %0.2f (+/- %0.2f)" % (scores_cv_accuracy.mean(), scores_cv_accuracy.std() * 2))  ### eduardo
print("Precision: %0.2f (+/- %0.2f)" % (scores_cv_precision.mean(), scores_cv_precision.std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (scores_cv_recall.mean(), scores_cv_recall.std() * 2))
print("F1: %0.2f (+/- %0.2f)" % (scores_cv_f1.mean(), scores_cv_f1.std() * 2))

Accuracy: 0.75 (+/- 0.07)
Precision: 0.70 (+/- 0.18)
Recall: 0.52 (+/- 0.12)
F1: 0.59 (+/- 0.10)


#### Como avaliar os vários conjuntos?

Na estratégia da validação cruzada, podemos imaginar como se fossem várias etapas de treino e teste, em que cada execução é uma partição diferente da base de dados. Nesse caso, temos como resultado não apenas uma medição, mas sim tantas quantas forem o número de partições selecionadas. Ao invés de olhar separadamente cada uma delas, tipicamente usamos a média de cada métrica sobre todos os conjuntos. Assim, temos uma ideia geral da performance do algoritmo, considerando todo o processo de validação. 

Além disso, é interessante também olha o desvio padrão desse conjunto de medidas. Se temos um desvio muito grande, ou seja, alguns conjuntos deram bons resultados e outros um resultado ruim, isso pode ser um indicativo de algo deve ser repensado. Podemos, por exemplo, querer penalizar um algoritmo se isso acontece. Ainda, pode ser interessante avaliar se existe alguma característica no dado em que a separação puramente aleatória não seja suficiente.


#### Quantos conjuntos de validação usar?

No exemplo anterior usamos uma validação cruzada com 5 conjuntos de validação, evidenciada pelo parâmetro cv=5. Não existe um número ideal definido ou uma regra exata para ajudar a escolher esse parâmetro. 

Frequentemente se usa 3, 5 e 10 como boas opções. Nada impede que sejam usados números maiores ou menores, o importante aqui é objetivo: avaliar a capacidade de generalização do método. 

Dois fatores importantes a serem levados em consideração ao escolher o número de conjuntos de validação:

* número de amostras disponíveis na base de dados: se nossa base de dados é muito pequena (como no nosso exemplo de hoje, menos de 1000 amostras), separar em poucos conjuntos pode ser ruim. Quanto menos conjuntos, maior o tamanho de cada um deles e consequentemente menos dado para treinamento do modelo.
* complexidade computacional do algoritmo escolhido: se o algoritmo escolhido é muito custoso computacionalmente, podemos não querer escolher um número de conjuntos muito grande. Isso aumentaria muito o número de execuções para um algoritmo que já é custoso. Isso pode ser especialmente relevante se a base de dados é muito grande. 

Vamos refazer o nosso exemplo, mas dessa vez considerando cv=10.

In [80]:
classifier_cv = RandomForestClassifier(n_estimators= 10, random_state=42)

scores_cv = cross_val_score(classifier_cv, data, labels, cv=10) ##### accuracy é o padrão
scores_cv_precision = cross_val_score(classifier_cv, data, labels, cv=10, scoring='precision')
scores_cv_recall = cross_val_score(classifier_cv, data, labels, cv=10, scoring='recall')
scores_cv_f1 = cross_val_score(classifier_cv, data, labels, cv=10, scoring='f1')

print("Acurácia: %0.2f (+/- %0.2f)" % (scores_cv.mean(), scores_cv.std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (scores_cv_precision.mean(), scores_cv_precision.std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (scores_cv_recall.mean(), scores_cv_recall.std() * 2))
print("F1: %0.2f (+/- %0.2f)" % (scores_cv_f1.mean(), scores_cv_f1.std() * 2))

Acurácia: 0.76 (+/- 0.09)
Precision: 0.74 (+/- 0.24)
Recall: 0.54 (+/- 0.14)
F1: 0.61 (+/- 0.12)


In [81]:
scores_cv

array([0.7012987 , 0.80519481, 0.72727273, 0.7012987 , 0.77922078,
       0.77922078, 0.81818182, 0.81818182, 0.73684211, 0.77631579])

In [82]:
scores_cv_precision

array([0.59090909, 0.92857143, 0.625     , 0.57692308, 0.91666667,
       0.75      , 0.84210526, 0.7826087 , 0.65      , 0.71428571])

In [83]:
df.shape  #### apenas 768 linhas

(768, 8)

Legal! Obtivemos uma pequena melhora somente aumentando um pouco o número de conjuntos de validação. Um possível motivo é que, com um número maior de conjuntos, temos menos exemplos em cada conjunto de teste e portanto mais amostras para o treinamento. Isso pode favorecer a construção do modelo e consequentemente ter melhores resultados. Se nossa base de dados fosse maior, talvez não teríamos observado esse efeito.


#### Lembrete: Obtendo predições individuais

Além de avaliar a performance dos algoritmos usando a validação cruzada, podemos estar interessados também em obter as predições propriamente. Para isso, podemos usar a função <a href = https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html> cross_val_predict </a>.

Essa função é útil em dois cenários:
* Quando queremos visualizar as previsões de alguma forma
* Quando queremos usar as previsões para alimentar algum modelo posterior


## Variações na Validação Cruzada

Até agora vimos uma estratégia de validação cruzada que está entre as mais usadas. Frequentemente essa estratégia é chamada de k-fold, em uma referência a escolha de k conjuntos para particionar o dado. 

Existem porém alguns cenários em que a estratégia vista até agora pode apresentar alguns problemas. Imagine por exemplo o cenário em que temos um desbalanceamento entre as classes da ordem de 90% de positivos e 10% negativos. Assim, uma seleção aleatória simples pode acabar não sendo adequada, uma vez que podemos ter poucas amostras da classe negativa no teste. Isso também ocorre no cenário multiclasse, especialmente se as classes tiverem prevalências muito diferentes. 

Para resolver esses casos, existem algumas variações possíveis para validação cruzada. Uma variação muito comumente usada é a validação cruzada estratificada. Nesta variação, a separação entre os conjuntos de treino e teste continua sendo feita aleatoriamente, porém ela preserva a proporção de cada classes nos conjuntos de treino e teste. Assim, se temos um cenário de desbalanceamento, como o descrito anteriormente, em que uma classe tem 90% de prevalência e outra 10%, cada um dos conjuntos criados para validação cruzada irá manter essa proporção.

Vamos fazer um exemplo da validação cruzada com estratificação!

O sklearn nos permite usar uma estrutura de iterables para facilitar o processo. Assim, ao invés de passar para a função cross_val_score o número de conjuntos de validação que queremos, podemos passar essa estrutura que vai construir a validação no formato desejado.

No caso da validação cruzada estratificada, vamos usar a função StratifiedKFold.

In [84]:
from sklearn.model_selection import StratifiedKFold

classifier_cv = RandomForestClassifier(n_estimators= 10, random_state=42)

cv_strat = StratifiedKFold(n_splits = 10)

scores_cv_strat = cross_val_score(classifier_cv, data, labels, cv=cv_strat)
scores_cv_strat_precision = cross_val_score(classifier_cv, data, labels, cv=cv_strat, scoring='precision')
scores_cv_strat_recall = cross_val_score(classifier_cv, data, labels, cv=cv_strat, scoring='recall')
scores_cv_strat_f1 = cross_val_score(classifier_cv, data, labels, cv=cv_strat, scoring='f1')

print("Acurácia: %0.2f (+/- %0.2f)" % (scores_cv_strat.mean(), scores_cv_strat.std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (scores_cv_strat_precision.mean(), scores_cv_strat_precision.std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (scores_cv_strat_recall.mean(), scores_cv_strat_recall.std() * 2))
print("F1: %0.2f (+/- %0.2f)" % (scores_cv_strat_f1.mean(), scores_cv_strat_f1.std() * 2))

Acurácia: 0.76 (+/- 0.09)
Precision: 0.74 (+/- 0.24)
Recall: 0.54 (+/- 0.14)
F1: 0.61 (+/- 0.12)


Como nossa base de dados é mais comportada, não observamos uma variação muito grande no resultado. Porém, essa estratégia pode ser fundamental para avaliar corretamente um modelo, no cenário em que os dados sejam mais desbalanceados e especialmente com multiclasse. 

### Reduzindo ainda mais o viés de seleção

Uma das maiores desvantagens que falamos sobre o método de treino-teste é o seu possível viés de seleção, que nos impossibilita de tirar conclusões sobre a capacidade de generalização do algoritmo avaliado.

A validação cruzada surge como uma excelente opção nesse cenário. Ao invés de ter um único conjunto de treino e teste, temos vários e mais do que isso, todos as amostras da base em algum momento são avaliadas no teste. 

Contudo, ainda assim temos um pequeno viés. Uma vez que as partições são criadas, a avalição é feita em cima dessas partições. Como dito anteriormente, sempre olhamos para a média da métrica sobre os vários conjuntos, além do desvio padrão. 

Uma maneira ainda mais conservadora de avaliar a validação cruzada é realizar esse processo repetidamente. Assim, os conjuntos são aleatorizados várias vezes e ao final temos uma medida ainda mais assertiva da capacidade do método. 

O sklearn possui um iterable para esse cenário também que você pode testar, como RepeatedKFold e RepeatedStratifiedKFold

## Leave One Out

Finalmente chegamos na nossa última estratégia de validação, chamada de Leave One Out (LOO). Como já falamos anteriormente, essa estratégia leva a validação cruzada ao máximo, fazendo com que cada exemplo da base de dados seja uma partição de teste. 

Para usar essa estratégia, temos algumas opções diferentes no sklearn

* Fazer cv = número de amostras da base
* Usar um iterable KFold e escolher o número de splits igual ao número de amostras da base de dados
* Usar um iterable específico para essa estratégias

Para o nosso exemplo, vamos escolher o último caso.

In [85]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut

classifier_cv = RandomForestClassifier(n_estimators= 10, random_state=42)

loo = LeaveOneOut()

scores_loo = cross_val_score(classifier_cv, data, labels, cv=loo)

print("Acurácia: %0.2f (+/- %0.2f)" % (scores_loo.mean(), scores_loo.std() * 2))

Acurácia: 0.75 (+/- 0.87)


In [ ]:
scores_loo

array([0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1.,
       0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1.,
       0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
       1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 1.,
       1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1., 0.,
       1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1.,
       0., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1.,
       1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 1.,
       1., 1., 1., 1., 1.

Novamente, devido ao tamanho e comportamento da nossa base de dados, não observamos muita diferença.

Um ponto a se destacar sobre a estratégia de LOO é que ela é computacionalmente custosa. Além disso, algumas métricas como precision e recall não são bem definidas, já que temos apenas uma amostra no teste. 

**Tipicamente, durante a seleção de modelos se usa a validação cruzada, tomando cuidados para casos de contorno (exemplo: dados desbalanceados) e com número de conjuntos 5 ou 10.**